2151131 朱沙桐

## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal(
            [784, 98]), trainable=True, dtype=tf.float32)
        self.b1 = tf.Variable(tf.zeros([98]), trainable=True, dtype=tf.float32)
        self.W2 = tf.Variable(tf.random.normal(
            [98, 10]), trainable=True, dtype=tf.float32)
        self.b2 = tf.Variable(tf.zeros([10]), trainable=True, dtype=tf.float32)
        
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 784])
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [5]:
train_data, test_data = mnist_dataset()
for epoch in range(500):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 91.84845 ; accuracy 0.10211667
epoch 1 : loss 83.035614 ; accuracy 0.10275
epoch 2 : loss 76.28547 ; accuracy 0.10446667
epoch 3 : loss 71.08835 ; accuracy 0.10538334
epoch 4 : loss 66.96749 ; accuracy 0.10718333
epoch 5 : loss 63.6277 ; accuracy 0.109066665
epoch 6 : loss 60.84216 ; accuracy 0.1117
epoch 7 : loss 58.489086 ; accuracy 0.11375
epoch 8 : loss 56.47683 ; accuracy 0.11655
epoch 9 : loss 54.735508 ; accuracy 0.1195
epoch 10 : loss 53.21015 ; accuracy 0.12261666
epoch 11 : loss 51.85543 ; accuracy 0.12598333
epoch 12 : loss 50.639515 ; accuracy 0.13011667
epoch 13 : loss 49.53517 ; accuracy 0.13463333
epoch 14 : loss 48.52185 ; accuracy 0.13966666
epoch 15 : loss 47.583923 ; accuracy 0.14446667
epoch 16 : loss 46.707985 ; accuracy 0.14996667
epoch 17 : loss 45.884857 ; accuracy 0.15493333
epoch 18 : loss 45.106407 ; accuracy 0.16051666
epoch 19 : loss 44.3656 ; accuracy 0.16578333
epoch 20 : loss 43.656944 ; accuracy 0.17101666
epoch 21 : loss 42.97686 ; accur